In [1]:
import findspark
findspark.init("/opt/spark/spark-3.4.0-bin-hadoop3")

In [2]:
from pyspark import SparkContext,SparkConf
from pyspark.streaming import StreamingContext
import sys,re
import random

In [10]:
random.seed(15)
conf = (SparkConf().setMaster("local[2]").setAppName("TextUpdater").set("spark.executor.memory","2g"))
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc,5)
ssc.checkpoint("file:///opt/spark/check")
sc.setLogLevel("WARN")



In [8]:
directory = "file:///opt/spark/data"

In [9]:
log_data = ssc.textFileStream(directory)

def mapIPValue(rdd):
    rdd1 = rdd.map(lambda a:a.split("-")[0]).map(lambda a:(a,1)).reduceByKey(lambda a,b:a+b).filter(lambda x:x[1]>30)
    return rdd1

transformed_access_log = log_data.transform(mapIPValue)
transformed_access_log.pprint(num=30)

In [6]:
ssc.start()

In [7]:
ssc.stop(stopSparkContext=True,stopGraceFully=True)